### IMPORTS

In [1]:
import pandas as pd
import numpy as np
import re

### DATA

In [2]:
wl_cams = pd.read_json('../data/WELAB/welab_cameras.json')
wl_lens = pd.read_json('../data/WELAB/welab_lenses.json')
wl_audio = pd.read_json('../data/WELAB/welab_audio.json')
wl_lights = pd.read_json('../data/WELAB/welab_lights.json')

In [3]:
rc_cams = pd.read_json('../data/RC SERVICE/rcservice_cameras.json')
rc_lenses = pd.read_json('../data/RC SERVICE/rcservice_lenses.json')

In [4]:
ov_cams = pd.read_json('../data/OVIDE/ovide_cameras.json')
ov_lights = pd.read_json('../data/OVIDE/ovide_lights.json')
ov_lenses = pd.read_json('../data/OVIDE/ovide_lenses.json')
ov_audio = pd.read_json('../data/OVIDE/ovide_audio.json')

In [9]:
rental_places = pd.read_json('../data/rental_places.json') 

### TRANSFORMATION

In [5]:
wl_cams

,CATEGORY,BRAND,NAME,PRICE a day,RENTAL,LINK
0,CAMERAS,PHANTOM,PHANTOM FLEX 4K 128GB,Pedir presupuesto,WELAB,https://welabplus.com/shop8/camaras/phantom/ph...
1,CAMERAS,GOPRO,GOPRO HERO 11 BLACK EDITION,85,WELAB,https://welabplus.com/shop8/camaras/gopro/gopr...
2,CAMERAS,GOPRO,GOPRO MAX 360,70,WELAB,https://welabplus.com/shop8/camaras/gopro/gopr...
3,CAMERAS,GOPRO,GOPRO HERO 10 BLACK EDITION,75,WELAB,https://welabplus.com/shop8/camaras/gopro/gopr...
4,CAMERAS,GOPRO,GOPRO HERO 9 BLACK EDITION,69,WELAB,https://welabplus.com/shop8/camaras/gopro/gopr...
...,...,...,...,...,...,...
56,CAMERAS,BLACKMAGIC,BLACKMAGIC MICRO CINEMA CAMERA,50,WELAB,https://welabplus.com/shop8/camaras/blackmagic...
57,CAMERAS,BLACKMAGIC,BLACKMAGIC CINEMA CAMERA 4K,65,WELAB,https://welabplus.com/shop8/camaras/blackmagic...
58,CAMERAS,KODAK,KODAK SUPER 8 mm,Pedir presupuesto,WELAB,https://welabplus.com/shop8/camaras/kodak/koda...
59,CAMERAS,SIGMA,SIGMA FP,50,WELAB,https://welabplus.com/shop8/camaras/camaras-fu...


In [6]:
ov_cams

,CATEGORY,TYPE,BRAND,NAME,PRICE a day,RENTAL,LINK
0,CAMERAS,Cine Digital,Arri,Cámara ARRI ALEXA 35 4K,Pedir presupuesto,OVIDE,https://www.ovide.com/alquiler-es/camara-arri-...
1,CAMERAS,Cine Digital,Arri,Cámara ARRI ALEXA LF,Pedir presupuesto,OVIDE,https://www.ovide.com/alquiler-es/camara-arri-...
2,CAMERAS,Cine Digital,Arri,Cámara Arri Alexa Mini,Pedir presupuesto,OVIDE,https://www.ovide.com/alquiler-es/arri-alexa-m...
3,CAMERAS,Cine Digital,Arri,Cámara ARRI ALEXA MINI LF,Pedir presupuesto,OVIDE,https://www.ovide.com/alquiler-es/camara-arri-...
4,CAMERAS,Cine Digital,Arri,Cámara Arri Alexa SXT Plus 4:3,Pedir presupuesto,OVIDE,https://www.ovide.com/alquiler-es/arri-alexa-s...
5,CAMERAS,Cine Digital,Canon,Cámara Canon C300 MKII PL,Pedir presupuesto,OVIDE,https://www.ovide.com/alquiler-es/canon-c300-m...
6,CAMERAS,Cine Digital,Canon,Cámara Canon C300 MKIII,Pedir presupuesto,OVIDE,https://www.ovide.com/alquiler-es/camara-canon...
7,CAMERAS,Cine Digital,Red,Cámara RED DSMC2 HELIUM 8K S35,Pedir presupuesto,OVIDE,https://www.ovide.com/alquiler-es/camara-red-d...
8,CAMERAS,Cine Digital,Red,Cámara RED DSMC2 MONSTRO 8K VV,Pedir presupuesto,OVIDE,https://www.ovide.com/alquiler-es/red-dsmc2-mo...
9,CAMERAS,Cine Digital,Red,Cámara RED KOMODO 6K S35,Pedir presupuesto,OVIDE,https://www.ovide.com/alquiler-es/camara-red-k...


In [7]:
rc_cams

,CATEGORY,TYPE,BRAND,NAME,PRICE a day,RENTAL,LINK
0,CAMERAS,Analogica,ARRI,Packs Arricam LT,Pedir presupuesto,RC SERVICE,http://www.rcservice.es/es/camaras-35mm-16mm/2...
1,CAMERAS,Analogica,MOVICAM,Packs Movicam Compact 35,Pedir presupuesto,RC SERVICE,http://www.rcservice.es/es/camaras-35mm-16mm/2...
2,CAMERAS,Analogica,ARRI,Packs Arri 435,Pedir presupuesto,RC SERVICE,http://www.rcservice.es/es/camaras-35mm-16mm/2...
3,CAMERAS,Analogica,ARRI,Packs Arri SR3,Pedir presupuesto,RC SERVICE,http://www.rcservice.es/es/camaras-35mm-16mm/2...
4,CAMERAS,Digital,ARRI,ARRI ALEXA 35,Pedir presupuesto,RC SERVICE,http://www.rcservice.es/es/alquiler-camaras-pe...
5,CAMERAS,Digital,ARRI,ARRI ALEXA MINI LF,Pedir presupuesto,RC SERVICE,http://www.rcservice.es/es/alquiler-camaras-pe...
6,CAMERAS,Digital,ARRI,ARRI ALEXA MINI,Pedir presupuesto,RC SERVICE,http://www.rcservice.es/es/alquiler-camaras-pe...
7,CAMERAS,Digital,ARRI,ARRI ALEXA XT,Pedir presupuesto,RC SERVICE,http://www.rcservice.es/es/alquiler-camaras-pe...
8,CAMERAS,Digital,ARRI,ARRI ALEXA PLUS,Pedir presupuesto,RC SERVICE,http://www.rcservice.es/es/alquiler-camaras-pe...
9,CAMERAS,Digital,ARRI,ARRI AMIRA PREMIUM,Pedir presupuesto,RC SERVICE,http://www.rcservice.es/es/alquiler-camaras-pe...


In [11]:
rental_places

,rental_place_id,rental_place_name,address,email,phone,website
0,1,WELAB PROFESSIONAL EQUIPMENT,"C. de Albasanz, 59, 28037, Madrid",info@welab.es,0034 911823355,https://www.welabplus.com
1,2,RC SERVICE,"Avda. Prado del Espino, 1, 28660, Boadilla del...",info@rcservice.es,0034 911855868,https://www.rcservice.es
2,3,OVIDE,"Av. Montes de Oca, 19, Nave 23, 28703, San Seb...",madrid@ovide.com,0034 915670407,https://www.ovide.com
